# Function Calling

(現在調整中)OpenAIが提供するAssistant APIのうちの一つの機能．関数のレスポンスを取得し回答を出力する．
- ドキュメント:https://platform.openai.com/docs/assistants/tools/function-calling
- 参考:https://note.com/npaka/n/nc3713dba5df6

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/nlp/llm_framework/function_calling.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [2]:
from openai import OpenAI
import os
import json

client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


### 入力の設定

In [3]:
instruction = "あなたはお天気ボットです。 提供されている関数を使用して質問に答えます。"
model = "gpt-4-1106-preview"
content = "群馬県の現在の気温を教えてください。"

# 関数の名前と説明文を定義
function_name = "getCurrentTemperature"
function_description = "特定の場所の現在の気温を取得"


In [4]:
assistant = client.beta.assistants.create(
    instructions = instruction,
    model = model,
    tools=[{
        "type": "function",
        "function": {
            "name": function_name,
            "description": function_description,
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "場所 (例:東京)"},
                    "unit": {"type": "string", "enum": ["c", "f"]}
                },
                "required": ["location"]
            }
        }
    }]
)

# スレッドの準備
thread = client.beta.threads.create()


# メッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = content
)



In [5]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)


user : 群馬県の現在の気温を教えてください。


### Function Callingの実行

In [6]:
# Function Callingの実行
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Function Callingの準備状態の確認
# 「in_progress」状態であれば関数の実行結果を受け取れる
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
print(run.status)


in_progress


### 関数の実行結果として何を渡せばいいかを確認
* ここでは「location」と「uint」を渡せばいいことが分かる

In [7]:
# require_actionのパラメータの取得
tool_id = run.required_action.submit_tool_outputs.tool_calls[0].id
tool_function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
tool_function_arguments = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)

print("id:", tool_id)
print("name:", tool_function_name)
print("arguments:", tool_function_arguments)


AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

### 自作の関数を実行する

In [ ]:
def getCurrentTemperature(location, unit):
    print("location:", location)
    print("unit:", unit)
    return "22度"

# 特定の場所の天気を取得
tool_function_output = getCurrentTemperature(
    tool_function_arguments["location"],
    tool_function_arguments["unit"]
)


### 関数の実行結果をAPI経由で提出
- 提出は10分以内

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_id,
            "output": tool_function_output,
        }
    ]
)


### 出力結果の確認

In [11]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)


user : 群馬県の現在の気温を教えてください。
assistant : 群馬県の現在の気温は22度です。
